# Neural Networks

![neuron](https://cdn-images-1.medium.com/max/1600/1*eBMwpBBboAXgqsawwOKkPw.png)

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
import keras

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/Pokemon.csv', index_col=0) #пропишите путь до файла

In [0]:
data = data._get_numeric_data().dropna(axis=1)

In [0]:
data.head()

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop('Legendary', axis=1), 
                                                    data['Legendary'], test_size=0.3, 
                                                    random_state=42)

In [0]:
X_train.shape

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

import keras
from keras.models import Sequential
from keras.layers import Dense

# Инициализируем ANN
classifier = Sequential()

# Добавим входной слой и первый внутренний (скрытый) слой
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu', input_dim = 8))

# Добавим второй внутренний слой
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))

# Добавим выходной слой
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Скомпилируем ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Обучим ANN на тренировочном датасете
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


In [0]:
from sklearn.metrics import roc_auc_score
# Сделаем предсказания и оценим нашу модель

y_pred = classifier.predict_proba(X_test)
roc_auc_score(y_test, y_pred)

# ---------------------------------------------------------------------

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/Pokemon.csv', index_col=0) 
data = data._get_numeric_data().dropna(axis=1).reset_index(drop=True)

target = data['Legendary']*1

In [0]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [0]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
val_rate = []
for tr_ind, val_ind in kf.split(data):
    lr = LogisticRegression(C=2, max_iter=1000,
                            warm_start=True, random_state=42)
    train = data.drop('Legendary', axis=1).loc[tr_ind]
    val = data.drop('Legendary', axis=1).loc[val_ind]

    target_train = target[tr_ind]
    target_val = target[val_ind]

    lr.fit(train, target_train)
    pred_i = lr.predict_proba(val)[:, 1]
    val_rate.append(roc_auc_score(target_val, pred_i))

In [0]:
print('Среднее: {:.3f}\nCтандартное отклонение: {:.3f}'.format(np.mean(val_rate), np.std(val_rate)))

In [0]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
val_rate = []
for tr_ind, val_ind in kf.split(data):
    train = data.drop('Legendary', axis=1).loc[tr_ind].reset_index(drop=True)
    val = data.drop('Legendary', axis=1).loc[val_ind].reset_index(drop=True)

    target_train = target[tr_ind].reset_index(drop=True)
    target_val = target[val_ind].reset_index(drop=True)

    sc = StandardScaler()
    sc.fit(train)
    train = sc.transform(train)
    val = sc.transform(val)

    classifier = Sequential()
    classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu', input_dim = 8))
    classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    classifier.fit(train, target_train, batch_size = 10, epochs = 30, verbose=1)
    pred_i = classifier.predict_proba(val)
    val_rate.append(roc_auc_score(target_val, pred_i))
    


In [0]:
print('Среднее: {:.3f}\nCтандартное отклонение: {:.3f}'.format(np.mean(val_rate), np.std(val_rate)))